In [ ]:
import sklearn
import io
import datetime
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error


In [ ]:
!pip install --upgrade pandas

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/data.csv")

In [ ]:
beg = input("From:")
end = input("End:")

From:2021-02-25
End:2021-02-26


In [ ]:
df.iloc[-10:]

NameError: ignored

In [ ]:

print(df['Date'].dtypes)
ind = df[df['Date']==beg].index.values
print(ind)
df1 = df.iloc[:ind[0]]

object
[1199]


In [ ]:
df1

,Date,High,Low,Open,Close,Volume,Adj Close,MA,EMA,up_band,mid_band,low_band,RSI,MACD,macdsignal,macdhist,ATR,AD,ADX,Previous,diff,up
0,2016-05-20,23.857500,23.629999,23.660000,23.805000,128104000.0,22.276043,25.642625,25.005582,29.143642,25.642625,22.141609,31.360767,-0.914995,-0.547360,-0.367635,0.514107,4.327961e+07,32.462892,23.180000,0.255001,True
1,2016-05-23,24.297501,23.917500,23.967501,24.107500,152074400.0,22.559116,25.439750,24.833384,29.013020,25.439750,21.866480,31.571109,-0.940933,-0.626075,-0.314858,0.503776,-4.380546e+07,32.613012,23.197500,0.302500,True
2,2016-05-24,24.522499,24.209999,24.305000,24.475000,140560800.0,22.903015,25.227000,24.692586,28.764577,25.227000,21.689423,33.501670,-0.937967,-0.688453,-0.249514,0.503213,6.325366e+07,32.829455,23.355000,0.367500,True
3,2016-05-25,24.934999,24.527500,24.667500,24.905001,152675200.0,23.305397,24.982875,24.543530,28.390018,24.982875,21.575732,32.123679,-0.943103,-0.739383,-0.203720,0.498802,-4.127288e+07,33.035076,23.127501,0.430000,True
4,2016-05-26,25.182501,24.660000,24.920000,25.102501,225324800.0,23.490213,24.710875,24.357003,27.968442,24.710875,21.453308,29.117392,-0.979656,-0.787438,-0.192218,0.496862,-1.860641e+08,33.660422,22.584999,0.197500,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,2021-02-18,130.000000,127.410004,129.199997,129.710007,96856700.0,129.710007,133.517501,134.072509,143.689159,133.517501,123.345842,53.369998,2.048214,2.546475,-0.498261,3.435743,8.702969e+09,14.199286,133.940002,-1.129990,False
1195,2021-02-19,130.710007,128.800003,130.240005,129.869995,87668800.0,129.869995,134.057001,134.388460,143.841032,134.057001,124.272969,56.849963,2.074100,2.452000,-0.377900,3.435456,8.786553e+09,13.779259,137.389999,0.159988,True
1196,2021-02-22,129.720001,125.599998,128.009995,126.000000,103916400.0,126.000000,134.349001,134.614321,144.089601,134.349001,124.608400,56.045964,2.020489,2.365698,-0.345209,3.359183,8.798172e+09,13.382781,136.759995,-3.869995,False
1197,2021-02-23,126.709999,118.389999,123.760002,125.860001,158273000.0,125.860001,134.592001,134.832957,144.333549,134.592001,124.850452,56.201216,1.967426,2.286044,-0.318617,3.404724,8.865974e+09,12.868199,136.910004,-0.139999,False


In [ ]:

features = list(pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Features.csv")['Features'])
features

['MA',
 'EMA',
 'up_band',
 'mid_band ',
 'low_band',
 'MACD',
 'macdsignal',
 'macdhist',
 'ATR',
 'AD',
 'RSI',
 'ADX',
 'Previous']

In [ ]:
X_Cols = df[['MA','EMA','up_band','mid_band','low_band','MACD','macdsignal','macdhist','ATR','AD','RSI','ADX','Previous']]
Y_Cols = df['Close']
Y_Cols = Y_Cols.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state = 0)
clf = RandomForestClassifier(n_estimators = 400, min_samples_split = 3 ,oob_score = True, criterion = "gini", random_state = 0,bootstrap=True)
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

In [ ]:
pred = df.iloc[-1:]
single = pred[['MA','EMA','up_band','mid_band','low_band','MACD','macdsignal','macdhist','ATR','AD','RSI','ADX','Previous']]
val = scipy.sparse.csr_matrix(single.values)
print(clf.predict(val))

[119]


In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df.iloc[:10]



,Actual,Predicted
198,34,34
328,39,40
45,24,24
263,38,38
621,50,50
520,47,47
186,33,33
987,68,70
1107,121,124
1052,94,95


In [ ]:
print('Correct Prediction (%): ', accuracy_score(y_test, clf.predict(X_test), normalize = True) * 100.0)
print('Mean squared error: ', mean_squared_error(y_test, y_pred) )

Correct Prediction (%):  47.524752475247524
Mean squared error:  2.504950495049505


# Up or down


In [ ]:
X_Cols = df[['MA','EMA','up_band','mid_band','low_band','MACD','macdsignal','macdhist','ATR','AD','RSI','ADX','Previous']]
Y_Cols = df['up']
Y_Cols = Y_Cols.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state = 0)
clf = RandomForestClassifier(n_estimators = 500, min_samples_split = 3 ,oob_score = True, criterion = "gini", random_state = 0,bootstrap=True)
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)
print('Correct Prediction (%): ', accuracy_score(y_test, clf.predict(X_test), normalize = True) * 100.0)


Correct Prediction (%):  50.1639344262295


In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df.iloc[:10]

,Actual,Predicted
1144,1,0
990,0,1
478,1,0
161,1,0
1153,1,0
987,0,0
568,0,0
5,0,0
805,0,0
552,0,0


# Cross Validation


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [100,200,300,400,500]
min_samples_split = [2,3,4,5]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
              # 'max_features': max_features,
              # 'max_depth': max_depth,
               'min_samples_split': min_samples_split,
              # 'min_samples_leaf': min_samples_leaf,
               }

In [ ]:
CV = RandomizedSearchCV(estimator = clf, param_distributions = random_grid)
CV.fit(X_train, y_train)
print(CV.best_params_)

{'n_estimators': 500, 'min_samples_split': 3}


# Feature Importance


In [ ]:
features_importance = clf.feature_importances_
 
print("Feature ranking:")
for i, data_class in enumerate(features):
    print("{}. {} ({})".format(i + 1, data_class, features_importance[i]))


Feature ranking:
1. MA (0.08086484914341201)
2. EMA (0.09077124273065926)
3. up_band (0.07600578087734695)
4. mid_band  (0.08145021554244387)
5. low_band (0.07490050358488824)
6. MACD (0.067500113768765)
7. macdsignal (0.07203644096988801)
8. macdhist (0.0619433108268474)
9. ATR (0.0799585235062083)
10. AD (0.08686519111541723)
11. RSI (0.05797234746357605)
12. ADX (0.07840100376769954)
13. Previous (0.09133047670284804)
